In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 4.9MB/s 


In [0]:
import pandas as pd
link_hilstrom = 'https://drive.google.com/open?id=15osyN4c5z1pSo1JkxwL_N8bZTksRvQuU'
fluff, id = link_hilstrom.split('=')
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Hillstrom.csv')
hillstrom_df = pd.read_csv('Hillstrom.csv')

In [0]:
import pandas as pd
link_ = 'https://drive.google.com/open?id=1b8N7WtwIe2WmQJD1KL5UAy70K13MxwKj'
fluff, id = link_.split('=')
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Lalonde.csv')
lalonde_df = pd.read_csv('Lalonde.csv')

In [0]:
print(hillstrom_df[:5])
print(lalonde_df[:5])

   recency history_segment  history  ...  visit  conversion spend
0       10  2) $100 - $200   142.44  ...      0           0   0.0
1        6  3) $200 - $350   329.08  ...      0           0   0.0
2        7  2) $100 - $200   180.65  ...      0           0   0.0
3        9  5) $500 - $750   675.83  ...      0           0   0.0
4        2    1) $0 - $100    45.34  ...      0           0   0.0

[5 rows x 12 columns]
   treatment   age  education  black  ...  married  nodegree  RE75        RE78
0        1.0  37.0       11.0    1.0  ...      1.0       1.0   0.0   9930.0460
1        1.0  22.0        9.0    0.0  ...      0.0       1.0   0.0   3595.8940
2        1.0  30.0       12.0    1.0  ...      0.0       0.0   0.0  24909.4500
3        1.0  27.0       11.0    1.0  ...      0.0       1.0   0.0   7506.1460
4        1.0  33.0        8.0    1.0  ...      0.0       1.0   0.0    289.7899

[5 rows x 9 columns]


In [0]:
import csv
import json
import os
from os.path import isfile, join
from sklearn.model_selection import KFold, StratifiedKFold


def preprocess_data(df, dataset='hillstrom', verbose=True):
    """
    Preprocessing the dataset
     - Use one-hot encoding for categorical features
     - Check the name of the target variable and treatment variable
     - Drop the unused columns
     - Delete the unused data
    
    Args:
        df: A pandas.DataFrame which have all data of the dataset
        dataset: the name of the dataset
    Return:
        # I recommend to split into the dataframes of predictor variables, the 
        # target variable, and the treatment varaible
        # df_x: the dataframes of predictor variables
        # s_y: target variables
        # s_t: treatment variables
    """
    if dataset in ['hillstrom', 'email']:
        # For Hillstrom dataset, the ‘‘visit’’ target variable was selected
        #   as the target variable of interest and the selected treatment is 
        #   the e-mail campaign for women’s merchandise [1]
        # [1] Kane K, Lo VSY, Zheng J. True-lift modeling: Comparison of methods. 
        #    J Market Anal. 2014;2:218–238
    
        # Delete unused data: men's email cases should be removed
        df_x = df[df.segment != 'Mens E-Mail'].reset_index()
        # Assign Y for target (visit: 0, 1)
        s_y = df_x['visit']
        # Assign T for treatment (segment: Womens E-Mail, Mens E-Mail (not used), No E-Mail)
        s_t = (df_x['segment'] == 'Womens E-Mail').astype('int64')
        # Drop unused columns from X
        df_x = df_x.drop(columns=['conversion', 'spend', 'visit', 'segment', 'index'])
        # One-hot encoding for categorical features
        df_x = pd.get_dummies(df_x)

    elif dataset in ['criteo', 'ad']:
        raise NotImplementedError

    elif dataset in ['lalonde', 'job']:
        # Delete unused data: None
        df_x = df.reset_index()
        # Target variables (RE78: earnings in 1978)
        s_y = df_x['RE78']
        # Treatment variables (treatment: 0, 1)
        s_t = df_x['treatment']
        # Drop unused columns
        df_x = df_x.drop(columns=['treatment', 'RE78', 'index'])
        # One-hot encoding for categorical features
        df_x = pd.get_dummies(df_x)

    else:
        raise NotImplementedError

    return df_x, s_y, s_t

In [152]:
def print_preproccessing_data(df_x, s_y, s_t):
    print(df_x[:5])
    print(df_x.columns.values)
    print(df_x.shape)
    print(s_y[:5])
    print(s_t[:5])


hillstrom_df_x, hillstrom_s_y, hillstrom_s_t = preprocess_data(hillstrom_df, 'hillstrom')
print_preproccessing_data(hillstrom_df_x, hillstrom_s_y, hillstrom_s_t)

lalonde_df_x, lalonde_s_y, lalonde_s_t = preprocess_data(lalonde_df, 'lalonde')
print_preproccessing_data(lalonde_df_x, lalonde_s_y, lalonde_s_t)

"""
def print_preproccessing_data(df):
    print(df[:5])
    print(df.columns.values)
    print(df.shape)
    print(df['Y'][:5])
    print(df['T'][:5])


hillstrom_df_pre = preprocess_data(hillstrom_df, 'hillstrom')
print_preproccessing_data(hillstrom_df_pre)

lalonde_df_pre = preprocess_data(lalonde_df, 'lalonde')
print_preproccessing_data(lalonde_df_pre)
"""

   recency  history  mens  ...  channel_Multichannel  channel_Phone  channel_Web
0       10   142.44     1  ...                     0              1            0
1        6   329.08     1  ...                     0              0            1
2        7   180.65     0  ...                     0              0            1
3        2    45.34     1  ...                     0              0            1
4        6   134.83     0  ...                     0              1            0

[5 rows x 18 columns]
['recency' 'history' 'mens' 'womens' 'newbie'
 'history_segment_1) $0 - $100' 'history_segment_2) $100 - $200'
 'history_segment_3) $200 - $350' 'history_segment_4) $350 - $500'
 'history_segment_5) $500 - $750' 'history_segment_6) $750 - $1,000'
 'history_segment_7) $1,000 +' 'zip_code_Rural' 'zip_code_Surburban'
 'zip_code_Urban' 'channel_Multichannel' 'channel_Phone' 'channel_Web']
(42693, 18)
0    0
1    0
2    0
3    0
4    1
Name: visit, dtype: int64
0    1
1    0
2    1
3    1
4 

"\ndef print_preproccessing_data(df):\n    print(df[:5])\n    print(df.columns.values)\n    print(df.shape)\n    print(df['Y'][:5])\n    print(df['T'][:5])\n\n\nhillstrom_df_pre = preprocess_data(hillstrom_df, 'hillstrom')\nprint_preproccessing_data(hillstrom_df_pre)\n\nlalonde_df_pre = preprocess_data(lalonde_df, 'lalonde')\nprint_preproccessing_data(lalonde_df_pre)\n"

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def performance(pr_y1_ct1, pr_y1_ct0, y, ct, groups=10):
    """
    1. Split the total customers into the given number of groups
    2. Calculate the statistics of each segment
    
    Args:
        pr_y1_ct1: the series (list) of the customer's expected return
        pr_y1_ct0: the expected return when a customer is not treated
        y: the observed return of customers
        ct: whether each customer is treated or not
        groups: the number of groups (segments). Should be 5, 10, or 20
    Return:
        DataFrame:
            columns:
                'n_y1_ct1': the number of treated responders
                'n_y1_ct0': the number of not treated responders
                'r_y1_ct1': the average return of treated customers
                'r_y1_ct0': the average return of not treated customers
                'n_ct1': the number of treated customers
                'n_ct0': the number of not treated customers
                'uplift': the average uplift (the average treatment effect)
            rows: the index of groups
    """

    ### check valid arguments
    if groups not in [5, 10, 20]:
        raise Exception("uplift: groups must be either 5, 10 or 20")

    ### check for NAs.
    if pr_y1_ct1.isnull().values.any():
        raise Exception("uplift: NA not permitted in pr_y1_ct1")
    if pr_y1_ct0.isnull().values.any():
        raise Exception("uplift: NA not permitted in pr_y1_ct0")
    if y.isnull().values.any():
        raise Exception("uplift: NA not permitted in y")
    if ct.isnull().values.any():
        raise Exception("uplift: NA not permitted in ct")

    ### check valid values for ct
    if set(ct) != {0, 1}:
        raise Exception("uplift: ct must be either 0 or 1")

    ### check length of arguments
    if not (len(pr_y1_ct1) == len(pr_y1_ct0) == len(y) == len(ct)):
        raise Exception("uplift: arguments pr_y1_ct1, pr_y1_ct0, y and ct must all have the same length")

    ###############################
    ###     Do it yourself!     ###
    ###############################
    # Make group id to split
    group_id = (pd.Series(range(0, len(y))) / (len(y) / groups)).astype('int32')
    
    # Split input data frame by group id
    # tr: treated responder
    # cr: controlled responder
    df = pd.DataFrame(data={'tr': y & ct, 'cr': y & (ct == 0).astype('int64'), 'ct': ct, 'non_ct': ct == 0,
                            'exp_tr': pr_y1_ct1 & ct, 'exp_cr': pr_y1_ct0 & ct, 'group_id': group_id})
    df_group = df.groupby('group_id')
    
    # Get group data
    n_ct1 = df_group['ct'].sum()
    n_ct0 = df_group['non_ct'].sum()
    n_y1_ct1 = df_group['tr'].sum()
    n_y1_ct0 = df_group['cr'].sum()
    r_y1_ct1 = df_group['exp_tr'].mean()
    r_y1_ct0 = df_group['exp_cr'].mean()
    uplift = r_y1_ct1 - r_y1_ct0
    
    # Create output data frame by grouped input data frame
    return pd.DataFrame(data={'n_y1_ct1': n_y1_ct1, 'n_y1_ct0': n_y1_ct0, 'r_y1_ct1': r_y1_ct1, 'r_y1_ct0': r_y1_ct0,
                              'n_ct1': n_ct1, 'n_ct0': n_ct0, 'uplift': uplift})


def qini(perf, plotit=True):
    """
    Calculating the incremental gains (y-axis of Qini curve)
     - First, the cumulitative sum of the treated and the control groups are
      calculated with respect to the total population in each group at the
      specified decile
     - Afterwards we calculate the percentage of the total amount of people
      (both treatment and control) are present in each decile
    Args:
        perf: A return of the performance function (above)
        plotit: whether draw a plot or not
    Return:
        1. Qini value
        2. return or save the plot if plotit is True
    """
    
    ###############################
    ###     Do it yourself!     ###
    ###############################
    return ##


In [138]:
df = performance(hillstrom_df_pre['Y'], hillstrom_df_pre['Y'], hillstrom_df_pre['Y'], hillstrom_df_pre['T'], 10)
print(df)

          n_y1_ct1  n_y1_ct0  r_y1_ct1  r_y1_ct0  n_ct1   n_ct0  uplift
group_id                                                               
0              326       236  0.076347  0.076347   2165  2105.0     0.0
1              335       226  0.078473  0.078473   2112  2157.0     0.0
2              334       231  0.078238  0.078238   2127  2142.0     0.0
3              320       202  0.074941  0.074941   2182  2088.0     0.0
4              325       221  0.076130  0.076130   2118  2151.0     0.0
5              330       228  0.077301  0.077301   2158  2111.0     0.0
6              317       231  0.074239  0.074239   2114  2156.0     0.0
7              309       240  0.072382  0.072382   2204  2065.0     0.0
8              317       229  0.074256  0.074256   2120  2149.0     0.0
9              325       218  0.076130  0.076130   2087  2182.0     0.0


In [0]:
import itertools
import numpy as np

def parameter_tuning(fit_mdl, pred_mdl, data, search_space):
    """
    Given a model, search all combination of parameter sets and find
    the best parameter set
    
    Args:
        fit_mdl: model function
        pred_mdl: predict function of fit_mdl
        data:
            {
                "x_train": predictor variables of training dataset,
                "y_train": target variables of training dataset,
                "ct_train": treatment variables of training dataset,
                "x_test": predictor variables of test (usually, validation) dataset,
                "y_test": target variables of test (usually, validation) dataset,
                "ct_test": treatment variables of test (usually, validation) dataset,
            }
        search_space:
            {
                parameter_name: [search values]
            }
    Return:
        The best parameter set
    """
    
    ###############################
    ###     Do it yourself!     ###
    ###############################
    return ##

  
def wrapper(fit_mdl, pred_mdl, data)
    """
    General wrapper approach
    
    Args:
        fit_mdl: model function
        pred_mdl: predict function of fit_mdl
        data:
            {
                "x_train": predictor variables of training dataset,
                "y_train": target variables of training dataset,
                "ct_train": treatment variables of training dataset,
                "x_test": predictor variables of test (usually, validation) dataset,
                "y_test": target variables of test (usually, validation) dataset,
                "ct_test": treatment variables of test (usually, validation) dataset,
            }
    Return:
        (A list of best models, The list of dropped variables)
    """
    
    ###############################
    ###     Do it yourself!     ###
    ###############################
    return ##


In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold


def tma(x, y, ct, method=LogisticRegression, **kwargs):
    """Training a model according to the "Two Model Approach" 
    (a.k.a. "Separate Model Approach")
    The default model is General Linear Model (GLM)
    
    Source: "Incremental Value Modeling" (Hansotia, 2002)

    Args:
        x: A data frame of predictors.
        y: A binary response (numeric) vector.
        ct: A binary response (numeric) representing the treatment assignment
            (coded as 0/1).
        method: A sklearn model specifying which classification or regression
            model to use. This should be a method that can handle a 
            multinominal class variable.

    Return:
        Dictionary: A dictionary of two models. One for the treatment group, 
            one for the control group.

            {
                'model_treat': a model for the treatment group,
                'model_control': a model for the control group
            }

    """
    
    ###############################
    ###     Do it yourself!     ###
    ###############################
    return {
        'model_treat': method(random_state=1234, solver='newton-cg').fit(x[ct == 1], y[ct == 1]),
        'model_control': method(random_state=1234, solver='newton-cg').fit(x[ct == 0], y[ct == 0])
    }


def predict_tma(obj, newdata, **kwargs):
    """Predictions according to the "Two Model Approach" 
    (a.k.a. "Separate Model Approach")
    
    For each instance in newdata two predictions are made:
    1) What is the probability of a person responding when treated?
    2) What is the probability of a person responding when not treated
      (i.e. part of control group)?

    Source: "Incremental Value Modeling" (Hansotia, 2002)

    Args:
        obj: A dictionary of two models. 
            One for the treatment group, one for the control group.
        newdata: A data frame containing the values at which predictions
            are required.
    
    Return:
        DataFrame: A dataframe with predicted returns for when the customers
            are treated and for when they are not treated.
    """

    ###############################
    ###     Do it yourself!     ###
    ###############################
    df_x = newdata.drop(['X', 'Y'])
    s_y = newdata['Y']
    s_t = newdata['T']
    
    return obj['model_treat'](df_x, s_y, s_t) - obj['model_control'](df_x, s_y, s_t)

In [0]:

from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split

def main():
    models = {'tma': {'model': tma, 'predict': predict_tma}}

    search_space = {}
    mean_qini = 0
    std_qini = 0
    seed = 1234

    ### Load data ###
    df_x, s_y, s_t = preprocess_data(hillstrom_df, 'hillstrom')    
    # df_x, s_y, s_t = preprocess_data(lalonde_df, 'lalonde')
    skf = StratifiedKFold(n_splits=10, random_state=seed)

    for model in models:
        ### Cross validation ###
        
        ### 10 fold validation ###
        skf = StratifiedKFold(n_splits=5, random_state=seed)
        
        for train_index, test_index in skf.split(df_x, s_y):
            print(train_index, len(train_index))
            print(test_index, len(test_index))
        
        ### Variable selection (General wrapper approach) ###

        ### Parameter tuning ###

        print("Model: {}\n".format(model))
        print("Tuning space: \n")
        for key, val in search_space.items():
            print("    '{}': {}\n".format(key, val))
        print("Seed: {}\n".format(seed))
        print("Qini value: mean = {}, std = {}\n\n".format(mean_qini, std_qini))


In [163]:
main()

[ 8363  8371  8386 ... 42690 42691 42692] 34154
[   0    1    2 ... 8563 8564 8565] 8539
[    0     1     2 ... 42690 42691 42692] 34154
[ 8363  8371  8386 ... 17085 17086 17087] 8539
[    0     1     2 ... 42690 42691 42692] 34154
[16985 16990 17019 ... 25630 25631 25632] 8539
[    0     1     2 ... 42690 42691 42692] 34155
[25528 25531 25534 ... 34166 34167 34168] 8538
[    0     1     2 ... 34166 34167 34168] 34155
[34083 34086 34091 ... 42690 42691 42692] 8538
Model: tma

Tuning space: 

Seed: 1234

Qini value: mean = 0, std = 0




In [0]:
import csv
import json
import os
from os.path import isfile, join
from sklearn.model_selection import KFold, StratifiedKFold


def preprocess_data(df, dataset='hillstrom', verbose=True):
    # For Hillstrom dataset, the ‘‘visit’’ target variable was selected
    #   as the target variable of interest and the selected treatment is 
    #   the e-mail campaign for women’s merchandise [1]
    # [1] Kane K, Lo VSY, Zheng J. True-lift modeling: Comparison of methods. 
    #    J Market Anal. 2014;2:218–238
    dataset = dataset.lower()
    if dataset in ('hillstrom', 'email'):
        columns = df.columns
        for col in columns:
            if df[col].dtype != object:
                continue
            df = pd.concat(
                    [df, pd.get_dummies(df[col], 
                                        prefix=col, 
                                        drop_first=False)],
                    axis=1)
            df.drop([col], axis=1, inplace=True)

        df.columns = [col.replace('-', '').replace(' ', '_').lower()
                      for col in df.columns]
        df = df[df.segment_mens_email == 0]
        df.index = range(len(df))
        df.drop(['segment_mens_email', 
                 'segment_no_email', 
                 'conversion', 
                 'spend'], axis=1, inplace=True)

        y_name = 'visit'
        t_name = 'segment_womens_email'
    elif dataset in ['criteo', 'ad']:
        pass
    elif dataset in ['lalonde', 'job']:
        pass
    else:
        raise NotImplementedError
    
    df['Y'] = df[y_name]
    df.drop([y_name], axis=1, inplace=True)
    df['T'] = df[t_name]
    df.drop([t_name], axis=1, inplace=True)
    
    return df

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold


def ct_y_assign(y, ct):
    if y == 1 and ct == 1:
        return "TR"
    elif y == 0 and ct == 1:
        return "TN"
    elif y == 1 and ct == 0:
        return "CR"
    elif y == 0 and ct == 0:
        return "CN"
    else:
        return None


def y_assign(ct_y):
    if ct_y in ("TR", "TN"):
        return 1
    elif ct_y in ("CR", "CN"):
        return 0
    else:
        return None


def ct_assign(ct_y):
    if ct_y in ("TR", "CR"):
        return 1
    elif ct_y in ("TN", "CN"):
        return 0
    else:
        return None


def main():
    df = pd.read_csv('data/email/Hillstrom.csv')
    dataset = 'hillstrom'
    df = preprocess_data(df)
    Y = df['Y']
    T = df['T']
    x = df.drop(['Y', 'T'], axis=1)
    ct_y = pd.DataFrame({'Y': Y, 'T': T})\
             .apply(lambda row: ct_y_assign(row['Y'], row['T']), axis=1)
    if dataset == 'hillstrom':
        fold_gen = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234).split(x, ct_y)
    elif dataset == 'lalonde':
        fold_gen = KFold(n_splits=5, shuffle=True, random_state=1234).split(x)
    else:
        pass

    for idx, (train_index, test_index) in enumerate(fold_gen):
        x_train = x.reindex(train_index)
        x_test = x.reindex(test_index)
        if dataset == 'hillstrom':
            Y = ct_y.apply(y_assign)
            T = ct_y.apply(ct_assign)
        y_train = Y.reindex(train_index)
        y_test = Y.reindex(test_index)
        ct_train = T.reindex(train_index)
        ct_test = T.reindex(test_index)

        mdl = tma(x_train, y_train, ct_train)
        pred = predict_tma(mdl, x_test)
        print(pred)        
        
        ## performance()
        ## qini()



main()
